In [2]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 42.0 MB/s eta 0:00:00


In [1]:
import pennylane as qml
from pennylane import numpy as np
#from pennylane.optimize import NesterovMomentumOptimizer

#100 qubits required!!!!!!!!
num_qubits = 18
dev = qml.device("default.qubit")
def layer(layer_weights):
    for wire in range(num_qubits):
        qml.Rot(*layer_weights[wire], wires=wire)

    for wires in range(num_qubits-1):
        qml.CNOT([wires, wires+1])
    qml.CNOT([num_qubits-1,0])

def state_preparation(x):
    qml.BasisState(x, wires=list(range(num_qubits)))

@qml.qnode(dev)
def circuit(weights, x):
    state_preparation(x)

    for layer_weights in weights:
        layer(layer_weights)

    return qml.expval(qml.PauliZ(0))

In [2]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

def square_loss(labels, predictions):
    # We use a call to qml.math.stack to allow subtracting the arrays directly
    return np.mean((labels - qml.math.stack(predictions)) ** 2)

def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc

def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [3]:
def binary_to_bits(binary_list):
    return [[int(i,2) for i in str(binary_string)[:num_qubits]] for binary_string in binary_list]
c1 = np.array(binary_to_bits(np.load("machine1.npy")))
y1 = -np.ones(len(c1))
c2 = np.array(binary_to_bits(np.load("machine2.npy")))
c1 = np.concatenate((c1,c2))

y1 = np.concatenate((y1,np.ones(len(c2)))).astype(int)
def one_hot(y, num_classes):
    return np.eye(num_classes)[y]

# Convert labels to one-hot encoding
y_data_one_hot = one_hot(y1, 2)

In [4]:

indices = np.arange(len(y1))
np.random.shuffle(indices)
train_indices = indices[:1600]
test_indices = indices[1600:]
X_train = c1[train_indices]
y_train = y1[train_indices]

X_test = c1[test_indices]
y_test = y1[test_indices]
del c2, y1, c1, indices, train_indices, test_indices

In [5]:
np.random.seed(0)

num_layers = 2
weights_init = 0.01 * np.random.randn( num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)
y_train_one_hot = one_hot(y_train,2)
opt = qml.AdamOptimizer()
batch_size = 5
weights = weights_init
bias = bias_init
print(weights.shape, bias.shape)
for it in range(10):
    for j in range(1):
        # Update the weights by one optimizer step, using only a limited batch of data
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = np.array(X_train)[batch_index]
        Y_batch = y_train_one_hot[:,j][batch_index]
        weights, bias = opt.step(cost, weights, bias, X=X_batch, Y=Y_batch)

        # Compute accuracy
        predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]

        current_cost = cost(weights, bias, X_train, y_train)
        acc = accuracy(y_train, predictions)

        print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")

(2, 18, 3) ()
Iter:    1 | Cost: 1.9794926 | Accuracy: 0.5025000
Iter:    2 | Cost: 1.9678271 | Accuracy: 0.5025000
Iter:    3 | Cost: 1.9518590 | Accuracy: 0.5025000
Iter:    4 | Cost: 1.9316566 | Accuracy: 0.5025000
Iter:    5 | Cost: 1.9073264 | Accuracy: 0.5025000
Iter:    6 | Cost: 1.8790494 | Accuracy: 0.5025000
Iter:    7 | Cost: 1.8470826 | Accuracy: 0.5025000
Iter:    8 | Cost: 1.8109486 | Accuracy: 0.5025000
Iter:    9 | Cost: 1.7720239 | Accuracy: 0.5025000
Iter:   10 | Cost: 1.7305123 | Accuracy: 0.5025000


In [6]:
predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]
acc = accuracy(y_test, predictions)

In [9]:
print("Accuracy:", acc)

Accuracy: 0.5175
